In [ ]:
%matplotlib notebook

In [ ]:
import copy
import ctypes
import random
import numpy as np
import open3d as o3d

In [ ]:
libBB = np.ctypeslib.load_library(libname="libcalculatebb", 
                                loader_path="/home/himanshu/Downloads/cpp_projects/cpp-projects/test_mvbb/build/")

In [ ]:
def calculateBB(pc):
    result = np.zeros((5,3))
    libBB.calculateBB.argtypes = [np.ctypeslib.ndpointer(dtype=np.float64, ndim=pc.ndim, shape=pc.shape,
                                  flags='C_CONTIGUOUS'), ctypes.c_size_t,
                                  np.ctypeslib.ndpointer(dtype=np.float64, ndim=result.ndim, shape=result.shape,
                                  flags='C_CONTIGUOUS')]
    
    libBB.calculateBB.restype = ctypes.c_int

    s = libBB.calculateBB(pc, len(pc), result)

    return result

In [ ]:
tree_pnts = np.loadtxt('/home/himanshu/python_stuff/PointCloud_CNN/dgcnn/pytorch/tree_non_tree/tree1.txt', delimiter=' ')
tree_pnts.shape

In [ ]:
pcd = o3d.io.read_point_cloud("/home/himanshu/Downloads/bunny/reconstruction/bun_zipper.ply")
pcd = pcd.voxel_down_sample(0.005)

In [ ]:
# result = calculateBB(np.asarray(pcd.points))
result = calculateBB(tree_pnts)

In [ ]:
import sys,os,imp,re
import math
import matplotlib as mpl

import matplotlib.cm as cm
import matplotlib.pyplot as plt

import matplotlib.colors as colors
import matplotlib.cm as cmx
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
def plotCube(ax,minP = np.array([-1.0,-1.0,-1.0]), maxP=np.array([1.0,1.0,1.0]),
             trans= np.array([0.0,0.0,0.0]),rotationMatrix=np.diag([1,1,1])):
    from itertools import product, combinations
    r = [-1, 1]
    
    centerPos = (maxP + minP)/2.0;
    print(centerPos)
    extent = (maxP - minP)/2.0;
    
    points = np.array([(-1, -1, -1),
                     (-1, -1, 1),
                     (-1, 1, -1),
                     (-1, 1, 1),
                     (1, -1, -1),
                     (1, -1, 1),
                     (1, 1, -1),
                     (1, 1, 1)]);
    
    for s, e in combinations(points, 2):
        if np.sum(np.abs(s-e)) == r[1]-r[0]: # no diagonal lines
            p1 = np.array(s,dtype=float); p2 = np.array(e,dtype=float);
            #scale points
            p1*=extent; p2*=extent;
            #rotate and translate points
            p1 = rotationMatrix.dot(p1 + centerPos) + trans;
            p2 = rotationMatrix.dot(p2 + centerPos) + trans;
            ax.plot3D(*zip(p1,p2), color="b")

# def plotAxis(ax,centerPos,A_IK,plotAxisScale=1):
#         for i,c in zip([0,1,2],['r','g','b']):
#             I_eK_i = A_IK[:,i];
#             lines = list(zip(centerPos,plotAxisScale*I_eK_i+centerPos))
#             v = Arrow3D(*lines, mutation_scale=50, lw=1, arrowstyle="-|>", color=c);
#             ax.plot3D(*lines, color=c)
#             ax.add_artist(v);

# def plotAxis2d(ax,centerPos,u,v,plotAxisScale=1):
    
#     x = np.vstack((centerPos,plotAxisScale*u+centerPos))
#     y = np.vstack((centerPos,plotAxisScale*v+centerPos))
#     ax.plot(x.T[0],x.T[1],'r',lw=2)
#     ax.plot(y.T[0],y.T[1],'b',lw=2)
        
            
# from matplotlib.patches import FancyArrowPatch
# from mpl_toolkits.mplot3d import proj3d
# class Arrow3D(FancyArrowPatch):
#         def __init__(self, xs, ys, zs, *args, **kwargs):
#             FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
#             self._verts3d = xs, ys, zs
     
#         def draw(self, renderer):
#             xs3d, ys3d, zs3d = self._verts3d
#             xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
#             self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
#             FancyArrowPatch.draw(self, renderer)
            
# def axisEqual3D(ax):
#     extents = np.array([getattr(ax, 'get_{}lim'.format(dim))() for dim in 'xyz'])
#     sz = extents[:,1] - extents[:,0]
#     centers = np.mean(extents, axis=1)
#     maxsize = max(abs(sz))
#     r = maxsize/2
#     for ctr, dim in zip(centers, 'xyz'):
#         getattr(ax, 'set_{}lim'.format(dim))(ctr - r, ctr + r)

In [ ]:
fig = plt.figure("MVBBTest")
    
    
points = tree_pnts#np.asarray(pcd.points)
K_min = result[0,0:3]
K_max = result[1,0:3]
A_IK = result[2:5,0:3]
    
center = np.zeros((3,));
    
ax = Axes3D(fig)
ax.scatter(points.T[0],points.T[1],points.T[2],c='b')
    
plotCube(ax,K_min,K_max,center,A_IK) # A_IK = R_KI (rotation from I to K)
# plotAxis(ax,center,A_IK,1)
# plotAxis(ax,center,np.identity(3),0.5)
# axisEqual3D(ax)

In [ ]:
from scipy.spatial import ConvexHull

# Calculate the convex hull (i.e wrapping points) for the set of points
hull = ConvexHull(points)

print("Convex hull vertices: ",points[hull.vertices].shape[0])

# Plot it (red is the convex hull points, i.e the wrapping points and grey is the rest)
fig2 = plt.figure(2)
ax = fig2.subplots(1, 1, subplot_kw={'projection':'3d'})

# Plot all the points
ax.scatter(points[:, 0], points[:, 1], points[:, 2], s=2, c='grey', zorder=10)

# Plot the convex hull points in red
ax.scatter(points[hull.vertices, 0], 
           points[hull.vertices, 1], 
           points[hull.vertices, 2], 
           s=20, c='r', zorder=20)
ax.set_title('Convex Hull')

In [ ]:
chull = np.array([points[hull.vertices, 0], points[hull.vertices, 1], points[hull.vertices, 2]])
chull = chull.T

In [ ]:
cov = np.cov(chull.T)
eigval, eigvec = np.linalg.eig(cov)

In [ ]:
vec = eigvec[:,0]
ang = np.arccos(vec)

In [ ]:
from scipy.spatial.transform import Rotation as R

In [ ]:
rz = np.array([[0],[0],[1]])@vec.reshape(1,3)
rx = np.array([[1],[0],[0]])@eigvec[:,1].reshape(1,3)
ry = np.array([[0],[1],[0]])@eigvec[:,2].reshape(1,3)

In [ ]:
r = rx+rz+ry

In [ ]:
rot = R.from_euler('xyz', ang, degrees=False)
mat = rot.as_matrix()

In [ ]:
fig = plt.figure("CVHull")

K_min = chull.min(axis=0)
K_max = chull.max(axis=0)
A_IK = r.T
    
center = np.zeros((3,))
    
ax = Axes3D(fig)
ax.scatter(points.T[0],points.T[1],points.T[2],c='b')
    
plotCube(ax,K_min,K_max,center,A_IK) # A_IK = R_KI (rotation from I to K)
# plotAxis(ax,center,A_IK,1)
# plotAxis(ax,center,np.identity(3),0.5)
# axisEqual3D(ax)

In [ ]:
(K_max + K_min)/2

In [ ]:
p = np.array([1, 1, 1])
p = p*(K_max - K_min)/2

#rotate and translate points
p1 = (r.T).dot(p + (K_max + K_min)/2)

In [ ]:
p1

In [ ]:
from itertools import product, combinations
r_ = [-1, 1]

fig = plt.figure("Hull")

K_min = chull.min(axis=0)
K_max = chull.max(axis=0)
A_IK = r.T
    
center = np.zeros((3,))
    


centerPos = (K_max + K_min)/2.0;
extent = (K_max - K_min)/2.0;
    
points = np.array([(-1, -1, -1),
                    (-1, -1, 1),
                     (-1, 1, -1),
                     (-1, 1, 1),
                     (1, -1, -1),
                     (1, -1, 1),
                     (1, 1, -1),
                     (1, 1, 1)]);

ax = Axes3D(fig)
for s, e in combinations(points, 2):
    if np.sum(np.abs(s-e)) == r_[1]-r_[0]: # no diagonal lines
        p1 = np.array(s,dtype=float); p2 = np.array(e,dtype=float);
        #scale points
        p1*=extent; p2*=extent;
        #rotate and translate points
#         p1 = A_IK.dot(p1 + centerPos)
#         p2 = A_IK.dot(p2 + centerPos)
        p1 = p1 + centerPos
        p2 = p2 + centerPos
        ax.plot3D(*zip(p1,p2), color="b")

In [ ]:
np.linalg.det(A_IK.T)